In [2]:
import numpy as np
import os
import pandas as pd

from sklearn.model_selection import train_test_split

from scipy.stats import chi2_contingency


import tensorflow as tf
import tensorflow_federated as tff
from tensorflow import keras
from tensorflow.keras.callbacks import EarlyStopping

from keras.models import Sequential 
from keras.layers import Dense
from keras.initializers import GlorotUniform
from keras.initializers import HeUniform
from keras.layers import Dense, Dropout, BatchNormalization

#ignore warnings
import warnings
warnings.filterwarnings('ignore')
print('-'*25)

TEST_SIZE = 0.2
NUM_CLIENTS = 5
ACTIVE_CLIENTS = 5
BATCH_SIZE = 5
path = os.path.dirname(tff.__file__)
print(path)

2024-09-17 20:17:48.131092: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-09-17 20:17:48.131141: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-09-17 20:17:48.131190: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-09-17 20:17:48.146489: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


-------------------------
/home/ella/Documents/FL/venv-federated/lib/python3.9/site-packages/tensorflow_federated


In [3]:
# Lista delle GPU disponibili
gpus = tf.config.list_physical_devices('GPU')
print("GPUs disponibili: ", gpus)

# Verifica se TensorFlow utilizza la GPU
if gpus:
    print("TensorFlow sta usando la GPU")
else:
    print("TensorFlow non sta usando la GPU")

GPUs disponibili:  [PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]
TensorFlow sta usando la GPU


2024-09-17 20:17:51.361795: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-17 20:17:51.372474: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-17 20:17:51.372669: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [4]:
# Import del dataset e divisione in train e test
train_df = pd.read_csv('datasets/train.csv')
test_df = pd.read_csv('datasets/test.csv')

# Viene diviso il train set in train e validation set
train_df, val_df = train_test_split(train_df, test_size = TEST_SIZE, random_state = 42)

train_x = train_df.iloc[:,:-1]
train_y = train_df.iloc[:,-1]

val_x = val_df.iloc[:,:-1]
val_y = val_df.iloc[:,-1]

test_x = test_df.iloc[:,:-1]
test_y = test_df.iloc[:,-1]

In [5]:
# Funzione per calcolare la dipendenza tra feature categoriche con il chi-square test.
# Il test ha come ipotesi nulla che le due variabili siano indipendenti. Quindi con un p-value
# minore di 0.05 si rigetta l'ipotesi nulla e si accetta che le due variabili sono dipendenti.
def chi_square(f1, f2):
    crosstab_result=pd.crosstab(index=f1,columns=f2)
    return chi2_contingency(crosstab_result)[1]

In [6]:
# Strategia: Home_planet
# Si nota la forte correlazione tra group ed home planet/destination e quindi per i nan si possono assegnre i valori del gruppo
# (che saranno uguali alla moda). Restano problematici i gruppi composti da una sola persona, si introduce quindi una nuova categoria
# di home planet e destination per persona con origine o destinazione sconosciuta.

def mv_hp_d(dataset):
    print("-------------------------- HomePlanet -------------------------")

    # HomePlanet
    p = chi_square(dataset['HomePlanet'], dataset['Group'])
    print(f"Le feature HomePlanet e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                           else "Le feature Destination e Group non sono correlate")

    print(f"Numero di persone con HomePlanet nullo: {str(dataset['HomePlanet'].isna().sum())}")
    dataset['HomePlanet'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['HomePlanet'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['HomePlanet'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['HomePlanet'].transform(lambda x: x.fillna("unknown"))
    print(f"Numero di persone con HomePlanet nullo: {str(dataset['HomePlanet'].isna().sum())}")

    # Destination
    print("-------------------------- Destination ------------------------")
    p = chi_square(dataset['Destination'], dataset['Group'])
    print(f"Le feature Destination e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                           else "Le feature Destination e Group non sono correlate")
    
    print(f"Numero di persone con Destination nullo: {str(dataset['Destination'].isna().sum())}")
    dataset['Destination'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Destination'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Destination'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['Destination'].transform(lambda x: x.fillna("unknown"))
    print(f"Numero di persone con Destination nullo: {str(dataset['Destination'].isna().sum())}")

# Strategia: Age
# Si distinguono 3 casi di persone con Age nullo: persone sole, persone con famiglia che non spendono e persone con famiglia che spendono.
# Per assegnare l'età si usa la mediana del caso di appartenenza, immaginando che è meno probabile che una persona sola sia un bambino
# e che al contrario è più probabile che una persona che non spende, ma è in un gruppo lo sia.
def mv_age(dataset):
    print("----------------------------- Age -----------------------------")
    solo = dataset['Solo']==1
    family_no_spending = (dataset['Family_size']>1) & (dataset['No_spending']==1)
    family_spending = (dataset['Family_size']>1) & (dataset['No_spending']==0)
    
    print(f"Numero di persone con Age nullo: {str(dataset['Age'].isna().sum())}, di cui:")
    print(f"Persone sole: {str(dataset['Age'][solo].isna().sum())}")
    print(f"Persone con famiglia che non spendono: {str(dataset['Age'][family_no_spending].isna().sum())}")   
    print(f"Persone con famiglia che spendono: {str(dataset['Age'][family_spending].isna().sum())}")

    dataset['Age'][solo] = dataset[solo]['Age'].transform(lambda x: x.fillna(dataset[solo]['Age'].median()))
    dataset['Age'][family_no_spending] = dataset[family_no_spending]['Age'].transform(lambda x: x.fillna(dataset[family_no_spending]['Age'].median()))
    dataset['Age'][family_spending] = dataset[family_spending]['Age'].transform(lambda x: x.fillna(dataset[family_spending]['Age'].median()))

    print(f"Numero di persone con Age nullo: {str(dataset['Age'].isna().sum())}")   


# Strategia: Surname
# Si osserva che i gruppi sono per gran parte composti da persone con lo stesso cognome, quindi si può assegnare la moda del cognome del gruppo
# alle persone con cognome nullo. Per i gruppi composti da una sola persona si assegna cognome 'unknown'.
def mv_surname(dataset):
    print("------------------------------ Surname ------------------------")

    p = chi_square(dataset['Surname'], dataset['Group'])
    print(f"Le feature Surname e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                       else "Le feature Surname e Group non sono correlate")

    print(f"Numero di persone con Surname nullo: {str(dataset['Surname'].isna().sum())}")
    dataset['Surname'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Surname'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Surname'][(dataset['Solo']==1)] = dataset[(dataset['Solo']==1)].groupby('Group')['Surname'].transform(lambda x: x.fillna("Unknown"))
    print(f"Numero di persone con Surname nullo: {str(dataset['Surname'].isna().sum())}")

# Strategia: RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
# Si sostituiscono i valori nulli con la media di spesa delle altre categorie non nulle, dopo aver osservato che ogni persona ha almeno un valore
# di spesa non nullo. Ad esempio, se una persona ha spesa nulla per RoomService e ShoppingMall, ma ha spesa per FoodCourt e Spa, si assegna la media
# di FoodCourt e Spa per RoomService e ShoppingMall.
def mv_exp(dataset):
    print("------ RoomService, FoodCourt, ShoppingMall, Spa, VRDeck ------")
    print(f"Persone con servizi nulli: {str(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].isnull().any(axis=1).sum())}")
    # Per ogni persona con almeno un servizio nullo, si calcola la media delle spese non nulle
    dataset['RoomService'] = dataset['RoomService'].fillna(dataset[['FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['FoodCourt'] = dataset['FoodCourt'].fillna(dataset[['RoomService', 'ShoppingMall', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['ShoppingMall'] = dataset['ShoppingMall'].fillna(dataset[['RoomService', 'FoodCourt', 'Spa', 'VRDeck']].mean(axis=1))
    dataset['Spa'] = dataset['Spa'].fillna(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'VRDeck']].mean(axis=1))
    dataset['VRDeck'] = dataset['VRDeck'].fillna(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa']].mean(axis=1))
    print(f"Persone con servizi nulli: {str(dataset[['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']].isnull().any(axis=1).sum())}")

# Controllare se i gruppi hanno tutti lo stesso valore di cabin_side
def mv_cabins(dataset):
    print("------------------------- Cabin_side --------------------------")
    p = chi_square(dataset['Cabin_side'], dataset['Group'])
    print(f"Le feature Cabin_side e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                          else "Le feature Cabin_side e Group non sono correlate")
    print(f"Numero di persone con Cabin_side nullo: {str(dataset['Cabin_side'].isna().sum())}")
    dataset['Cabin_side'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Cabin_side'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Cabin_side'][(dataset['Solo']==1)] = dataset['Cabin_side'][(dataset['Solo']==1)].fillna('unknown')
    print(f"Numero di persone con Cabin_side nullo: {str(dataset['Cabin_side'].isna().sum())}")

    print("------------------------ Cabin_number -------------------------")
    p = chi_square(dataset['Cabin_number'], dataset['Group'])
    print(f"Le feature Cabin_number e Group sono correlate, con p-value{p}" if p < 0.05 \
                                                                            else "Le feature Cabin_number e Group non sono correlate")
    print(f"Numero di persone con Cabin_number nullo: {str(dataset['Cabin_number'].isna().sum())}")
    dataset['Cabin_number'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('Group')['Cabin_number'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Cabin_number'][(dataset['Solo']==1)] = dataset['Cabin_number'][(dataset['Solo']==1)].fillna(0)
    print(f"Numero di persone con Cabin_number nullo: {str(dataset['Cabin_number'].isna().sum())}")
    
    print("------------------------- Cabin_deck --------------------------")
    p = chi_square(dataset['Cabin_deck'], dataset['HomePlanet'])
    print(f"Le feature Cabin_deck e HomePlanet sono correlate, con p-value{p}" if p < 0.05 \
                                                                               else "Le feature Cabin_deck e HomePlanet non sono correlate")
    print(f"Numero di persone con Cabin_deck nullo: {str(dataset['Cabin_deck'].isna().sum())}")
    dataset['Cabin_deck'][(dataset['Solo']==0)] = dataset[(dataset['Solo']==0)].groupby('HomePlanet')['Cabin_deck'].transform(lambda x: x.fillna(x.mode()[0]))
    dataset['Cabin_deck'][(dataset['Solo']==1)] = dataset['Cabin_deck'][(dataset['Solo']==1)].fillna('unknown')
    print(f"Numero di persone con Cabin_deck nullo: {str(dataset['Cabin_deck'].isna().sum())}")

# Strategia: VIP, CrioSleep
# Per le feature VIP e CrioSleep si assegna il valore della moda a tutti i valori nulli.
def mv_vip_crio(dataset):
    print("----------------------------- VIP, CryoSleep ------------------")
    print(f"Numero di persone con VIP nullo: {str(dataset['VIP'].isna().sum())}")
    dataset['VIP'] = dataset['VIP'].fillna(dataset['VIP'].mode()[0])
    print(f"Numero di persone con VIP nullo: {str(dataset['VIP'].isna().sum())}")

    print(f"Numero di persone con CryoSleep nullo: {str(dataset['CryoSleep'].isna().sum())}")
    dataset['CryoSleep'] = dataset['CryoSleep'].fillna(dataset['CryoSleep'].mode()[0])
    print(f"Numero di persone con CryoSleep nullo: {str(dataset['CryoSleep'].isna().sum())}")


In [7]:

def preprocess_data(dataset):
    print(dataset.columns)
    # Viene aggiunta una colonna per contare il numero di persone nel gruppo di appartenenza e se la persona è da sola
    dataset['Group'] = dataset['PassengerId'].apply(lambda x: x.split('_')[0]).astype(int)
    dataset['Group_size']=dataset['Group'].map(lambda x: dataset['Group'].value_counts()[x])
    dataset['Solo']=(dataset['Group_size']==1).astype(int)

    # Viene creata una colonna per il cognome e si rimuove la colonna Name
    dataset['Name'].fillna('Unknown Unknown', inplace=True)
    dataset['Surname']=dataset['Name'].str.split().str[-1]
    dataset.loc[dataset['Surname']=='Unknown','Surname']=np.nan
    dataset.drop('Name', axis=1, inplace=True)

    # Vengono riempiti i valori nulli delle colonne RoomService, FoodCourt, ShoppingMall, Spa, VRDeck
    mv_exp(dataset)
    
    # Vengono prese tutte le colonne che rappresentano le spese e si aggiunge una colonna per contare il
    # totale speso dalla persona e una colonna booleana per indicare se la persona non ha speso nulla
    exp_feats=['RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck']
    dataset['Expenditure']=dataset[exp_feats].sum(axis=1)
    dataset['No_spending']=(dataset['Expenditure']==0).astype(int)

    # Vengono riempiti i valori nulli della colonna HomePlanet, Destination, Surmane
    mv_hp_d(dataset)
    mv_surname(dataset)

    # Viene aggiunta una colonna per contare le persone che hanno lo stesso cognome
    dataset['Family_size']=dataset[(dataset['Surname']!='Unknown')].groupby('Group')['Surname'].transform('count')
    dataset.loc[dataset['Surname']=='Unknown','Family_size']=1
    dataset.loc[dataset['Family_size']>100,'Family_size']=np.nan

    # Vengono riempiti i valori nulli della colonna Age
    mv_age(dataset)


    # Viene aggiunta una colonna che assegna ogni passeggero al gruppo di età di appartenenza
    dataset['Age_group']=0
    dataset.loc[dataset['Age']<=12,'Age_group']='Age_0-12'
    dataset.loc[(dataset['Age']>12) & (dataset['Age']<18),'Age_group']='Age_13-17'
    dataset.loc[(dataset['Age']>=18) & (dataset['Age']<=25),'Age_group']='Age_18-25'
    dataset.loc[(dataset['Age']>25) & (dataset['Age']<=30),'Age_group']='Age_26-30'
    dataset.loc[(dataset['Age']>30) & (dataset['Age']<=50),'Age_group']='Age_31-50'
    dataset.loc[dataset['Age']>50,'Age_group']='Age_51+'

    # Separazione della colonna 'Cabin' nelle colonne deck, number e side
    dataset['Cabin'].fillna('Z/9999/Z', inplace=True)

    dataset['Cabin_deck'] = dataset['Cabin'].apply(lambda x: x.split('/')[0])
    dataset['Cabin_number'] = dataset['Cabin'].apply(lambda x: x.split('/')[1]).astype(int)
    dataset['Cabin_side'] = dataset['Cabin'].apply(lambda x: x.split('/')[2])

    dataset.loc[dataset['Cabin_deck']=='Z', 'Cabin_deck']=np.nan
    dataset.loc[dataset['Cabin_number']==9999, 'Cabin_number']=np.nan
    dataset.loc[dataset['Cabin_side']=='Z', 'Cabin_side']=np.nan

    dataset.drop('Cabin', axis=1, inplace=True)
    mv_cabins(dataset)
    mv_vip_crio(dataset)


preprocess_data(train_x)
preprocess_data(val_x)

Index(['PassengerId', 'HomePlanet', 'CryoSleep', 'Cabin', 'Destination', 'Age',
       'VIP', 'RoomService', 'FoodCourt', 'ShoppingMall', 'Spa', 'VRDeck',
       'Name'],
      dtype='object')
------ RoomService, FoodCourt, ShoppingMall, Spa, VRDeck ------
Persone con servizi nulli: 691
Persone con servizi nulli: 0
-------------------------- HomePlanet -------------------------
Le feature HomePlanet e Group sono correlate, con p-value7.383208949075836e-101
Numero di persone con HomePlanet nullo: 168
Numero di persone con HomePlanet nullo: 0
-------------------------- Destination ------------------------
Le feature Destination e Group sono correlate, con p-value0.00016495054761429579
Numero di persone con Destination nullo: 139
Numero di persone con Destination nullo: 0
------------------------------ Surname ------------------------
Le feature Surname e Group sono correlate, con p-value0.0
Numero di persone con Surname nullo: 159
Numero di persone con Surname nullo: 0
------------------

In [8]:
from sklearn.preprocessing import OrdinalEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.compose import make_column_selector as selector

def feature_sel(dataset, label, fs, importance=0.03):
    # Train the Random Forest model
    dataset = dataset.apply(lambda col: col.astype(str) if col.dtype == 'object' else col)

    # Seleziona le colonne numeriche e categoriali
    num_cols = dataset.select_dtypes(exclude='object').columns
    cat_cols = dataset.select_dtypes(include='object').columns

    # Applica lo scaling ai dati numerici
    scaler = StandardScaler()
    dataset[num_cols] = scaler.fit_transform(dataset[num_cols])

    # Applica l'encoding ai dati categoriali
    encoder = OrdinalEncoder()
    dataset[cat_cols] = encoder.fit_transform(dataset[cat_cols])
    if fs:
        # Train the Random Forest model
        rf = RandomForestClassifier(n_estimators=150, random_state=42, n_jobs=-1, class_weight='balanced')
        rf.fit(dataset, label)

        # Extract feature importances
        importances = rf.feature_importances_
        feature_names = dataset.columns
        feature_importance_df = pd.DataFrame({'Feature': feature_names, 'Importance': importances})

        # Rank features by importance
        feature_importance_df = feature_importance_df.sort_values(by='Importance', ascending=False)
        print(feature_importance_df)

        # Select top N features (example selecting top 10 features)
        top_features = feature_importance_df['Feature'][(feature_importance_df['Importance']>=importance)].values
        return dataset[top_features]
    return dataset

train_x = feature_sel(train_x, train_y, True)
val_x = feature_sel(val_x, train_y, False)
val_x = val_x[train_x.columns]

train_x = pd.get_dummies(train_x, columns=train_x.select_dtypes(include=['object', 'bool']).columns.difference(['Surname', 'Cabin_number']))
val_x = pd.get_dummies(val_x, columns=val_x.select_dtypes(include=['object', 'bool']).columns.difference(['Surname', 'Cabin_number']))

train_x = train_x.astype(float)
train_y = train_y.astype(float)

val_x = val_x.astype(float)
val_y = val_y.astype(float)


         Feature  Importance
15   Expenditure    0.103269
20  Cabin_number    0.072006
0    PassengerId    0.070462
9            Spa    0.066563
11         Group    0.065693
14       Surname    0.064711
7      FoodCourt    0.062786
10        VRDeck    0.060425
4            Age    0.059240
2      CryoSleep    0.058959
16   No_spending    0.055042
8   ShoppingMall    0.050822
6    RoomService    0.049670
19    Cabin_deck    0.039829
1     HomePlanet    0.026782
18     Age_group    0.022704
21    Cabin_side    0.019293
3    Destination    0.018012
12    Group_size    0.013688
17   Family_size    0.013156
13          Solo    0.005624
5            VIP    0.001265


In [9]:
# Initialising the NN

model = Sequential()
EPOCHS = 1000
BATCH_SIZE = 750
DROPOUT = 0.1
np.random.seed(42)
tf.random.set_seed(42)

# layers
model.add(Dense(train_x.shape[-1], kernel_initializer = HeUniform(), activation = 'relu', input_dim = train_x.shape[-1]))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT))
model.add(Dense(1024, kernel_initializer = HeUniform(), activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT))
model.add(Dense(256, kernel_initializer = HeUniform(), activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT))
model.add(Dense(128, kernel_initializer = HeUniform(), activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
model.add(BatchNormalization())
model.add(Dropout(DROPOUT))
model.add(Dense(1, kernel_initializer = GlorotUniform(), activation = 'sigmoid'))

# summary
model.summary()
# Compiling the NN
initial_learning_rate = 0.01
final_learning_rate = 0.0001
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/EPOCHS)
steps_per_epoch = int(train_x.shape[0]/BATCH_SIZE)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=steps_per_epoch,
    decay_rate=learning_rate_decay_factor,
    staircase=True)
model.compile(optimizer = keras.optimizers.Adam(learning_rate=lr_schedule), loss = 'binary_focal_crossentropy', metrics = ['accuracy'])

# Train the NN
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_x, train_y,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=[early_stopping_cb],
                    validation_data=(val_x, val_y))

plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
plt.legend(['val_accuracy', 'accuracy'])
plt.show()

2024-09-17 20:18:02.677159: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-17 20:18:02.677486: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-09-17 20:18:02.677689: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 14)                210       
                                                                 
 batch_normalization (Batch  (None, 14)                56        
 Normalization)                                                  
                                                                 
 dropout (Dropout)           (None, 14)                0         
                                                                 
 dense_1 (Dense)             (None, 1024)              15360     
                                                                 
 batch_normalization_1 (Bat  (None, 1024)              4096      
 chNormalization)                                                
                                                                 
 dropout_1 (Dropout)         (None, 1024)              0

2024-09-17 20:18:03.220037: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


Epoch 1/1000


2024-09-17 20:18:07.175758: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x7cbde4164f20 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2024-09-17 20:18:07.175813: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce MX230, Compute Capability 6.1
2024-09-17 20:18:07.183674: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-09-17 20:18:07.239709: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8700
2024-09-17 20:18:07.347593: I ./tensorflow/compiler/jit/device_compiler.h:186] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


10/10 [==============================] - 5s 40ms/step - loss: 0.6195 - accuracy: 0.5013 - val_loss: 6.8848 - val_accuracy: 0.4945
Epoch 2/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.3069 - accuracy: 0.5027 - val_loss: 1.1047 - val_accuracy: 0.4940
Epoch 3/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.2870 - accuracy: 0.5111 - val_loss: 0.3773 - val_accuracy: 0.4894
Epoch 4/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.2814 - accuracy: 0.5079 - val_loss: 0.2869 - val_accuracy: 0.4888
Epoch 5/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.2738 - accuracy: 0.5102 - val_loss: 0.2782 - val_accuracy: 0.4951
Epoch 6/1000
10/10 [==============================] - 0s 16ms/step - loss: 0.2693 - accuracy: 0.5078 - val_loss: 0.2729 - val_accuracy: 0.4928
Epoch 7/1000
10/10 [==============================] - 0s 15ms/step - loss: 0.2629 - accuracy: 0.5014 - val_loss: 0.2776 - val_accuracy: 0.4830
Epoch 8/1000

NameError: name 'plt' is not defined

In [ ]:
# Initialising the NN
from keras.initializers import GlorotUniform
from keras.initializers import HeUniform
from keras.layers import Dense, Dropout, BatchNormalization
model = Sequential()
EPOCHS = 1000
BATCH_SIZE = 512
np.random.seed(42)
tf.random.set_seed(42)

# layers
model.add(Dense(train_x.shape[-1]*2, kernel_initializer = HeUniform(), activation = 'relu', input_dim = train_x.shape[-1]))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
model.add(Dense(16, kernel_initializer = HeUniform(), activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
model.add(Dense(32, kernel_initializer = HeUniform(), activation = 'relu', kernel_regularizer = tf.keras.regularizers.l2(30e-6)))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
#model.add(Dense(64, kernel_initializer = HeUniform(), activation = 'relu'))
#model.add(Dense(128, kernel_initializer = HeUniform(), activation = 'relu'))
#model.add(BatchNormalization())
#model.add(Dropout(0.5))
model.add(Dense(1, kernel_initializer = GlorotUniform(), activation = 'sigmoid'))

# summary
model.summary()
# Compiling the NN
initial_learning_rate = 0.001
final_learning_rate = 0.0001
learning_rate_decay_factor = (final_learning_rate / initial_learning_rate)**(1/EPOCHS)
steps_per_epoch = int(train_x.shape[0]/BATCH_SIZE)

lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate,
    decay_steps=steps_per_epoch,
    decay_rate=learning_rate_decay_factor,
    staircase=True)
model.compile(optimizer = keras.optimizers.Adam(learning_rate=lr_schedule), loss = 'binary_focal_crossentropy', metrics = ['accuracy'])

# Train the NN
early_stopping_cb = keras.callbacks.EarlyStopping(patience=20, min_delta=0.0001, restore_best_weights=True)

history = model.fit(train_x, train_y,
                    epochs=EPOCHS,
                    batch_size=BATCH_SIZE,
                    callbacks=[early_stopping_cb],
                    validation_data=(val_x, val_y))

In [ ]:
# stampare il grafico dell'accuracy e della loss tra tarining e validation

plt.plot(history.history['val_accuracy'])
plt.plot(history.history['accuracy'])
# aggiungere una legenda
plt.legend(['val_accuracy', 'accuracy'])
plt.show()

In [ ]:
from keras.initializers import HeNormal, GlorotUniform
model = keras.Sequential(
    [
        keras.layers.Dense(units=train_x.shape[-1], activation="relu", input_shape=(train_x.shape[-1],), kernel_initializer=GlorotUniform()),
        keras.layers.Dense(units=128, activation="relu", kernel_initializer=HeNormal()),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(units=64, activation="relu", kernel_initializer=HeNormal()),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(units=64, activation="relu", kernel_initializer=HeNormal()),
        keras.layers.Dense(units=32, activation="relu", kernel_initializer=HeNormal()),
        keras.layers.Dense(units=1, activation="sigmoid", kernel_initializer=GlorotUniform()),
    ]
)

learning_rate = 0.0001

early_stopping = EarlyStopping(
    min_delta = 0.0002, # minimium amount of change to count as an improvement
    patience  = 20,     # how many epochs to wait before stopping
    restore_best_weights=True,
)

model.compile(optimizer=keras.optimizers.Adam(learning_rate=learning_rate),
              loss="binary_crossentropy",
              metrics=['accuracy']
             )
history = model.fit(train_x, train_y,
          epochs=500,
          batch_size=512,
          callbacks=[early_stopping],
          validation_data=(val_x, val_y))

logs = pd.DataFrame(history.history)